# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [1]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk

nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

from newsapi.newsapi_client import NewsApiClient

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\watso\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
# Read your api key environment variable
# YOUR CODE HERE!
load_dotenv()

True

In [3]:
# Set News API Key
api_key=os.environ["NEWS_API_KEY"]
api_key

'efb7df5406ce4b94b58e9441f1d1e478'

In [4]:
# Create a newsapi client
# YOUR CODE HERE!
newsapi = NewsApiClient(api_key=api_key)
newsapi

In [5]:
# Fetch the Bitcoin news articles
# YOUR CODE HERE!
btc_news_en = newsapi.get_everything(
    q="bitcoin AND btc",
    language="en"
)

# Show the total number of news
btc_news_en["totalResults"]

1799

In [6]:
def create_df(news, language):
    articles = []
    for article in news:
        try:
            title = article["title"]
            description = article["description"]
            text = article["content"]
            date = article["publishedAt"][:10]

            articles.append({
                "title": title,
                "description": description,
                "text": text,
                "date": date,
                "language": language
            })
        except AttributeError:
            pass

    return pd.DataFrame(articles)

In [7]:
btc_news_df = create_df(btc_news_en["articles"], "en")
btc_news_df.head()

,title,description,text,date,language
0,"Who Bought $1.6B in Bitcoin Wednesday, and Why?",last week the cryptocurrency market persistent...,"Specifically, why did someone make a massive p...",2021-10-10,en
1,The Currency Of The Future: A Bitcoin (BTC) Boom,Bitcoin reaches a new all-time high as crypto-...,Following SEC Chair Gary Gensler's landmark ap...,2021-10-20,en
2,Crypto miners are hoarding bitcoin amid recent...,Long-term bitcoin holders just keep holding as...,Bitcoin. Sean Gladwell\r\nSean Gladwell\r\nCry...,2021-10-28,en
3,SEC Finally Greenlights Bitcoin Futures ETF Wi...,"Friday, October 16 saw the approval of a bitco...","Friday, October 16 saw the approval of a bitco...",2021-10-16,en
4,Bitcoin inflows hit record high so far in 2021...,Inflows into bitcoin products and funds have h...,"NEW YORK, Nov 8 (Reuters) - Inflows into bitco...",2021-11-08,en


In [8]:
# Fetch the Ethereum news articles
# YOUR CODE HERE!
eth_news_en = newsapi.get_everything(
    q="ethereum AND eth",
    language="en"
)

# Show the total number of news
eth_news_en["totalResults"]

637

In [9]:
eth_news_df = create_df(eth_news_en["articles"], "en")
eth_news_df.head()

,title,description,text,date,language
0,Nervos launches cross-chain bridge to connect ...,A new cross-chain bridge is currently connecte...,A new cross-chain bridge is currently connecte...,2021-10-16,en
1,Cryptocurrency ether hits all time high of $44...,Ether the world's second largest cryptocurren...,"The exchange rates and logos of Bitcoin (BTH),...",2021-10-29,en
2,Ethereum’s upgrade is finally coming — will it...,The world’s second most valuable cryptocurrenc...,The worlds second most valuable cryptocurrency...,2021-10-25,en
3,On the road of 100 Ethereum Icon Renderings — ...,On the road of 100 Ethereum Icon Renderings — ...,Taoliang Wang is a visual designer based in Ch...,2021-10-12,en
4,Wyoming Blockchain City Planned by Crypto Inve...,Blockchain entrepreneurs are now poised to bui...,Blockchain entrepreneurs are now poised to bui...,2021-11-10,en


In [10]:
## Sentiment function
########### Don't Use, below function is better and returns more data 
# def return_sentiment_df(news_articles):
#     empty_list = []
#     for article in news_articles["articles"]:
#         try:
#             text = article["content"]
#             date = article["publishedAt"][:10]
#             sentiment = analyzer.polarity_scores(text)
#             compound = sentiment["compound"]
#             pos = sentiment["pos"]
#             neu = sentiment["neu"]
#             neg = sentiment["neg"]

#             empty_list.append({
#                 "text": text,
#                 "date": date,
#                 "compound": compound,
#                 "positive": pos,
#                 "negative": neg,
#                 "neutral": neu

#             })

#         except AttributeError:
#             pass

#     # Create DataFrame
#     df = pd.DataFrame(empty_list)

#     # Reorder DataFrame columns
#     cols = ["date", "text", "compound", "positive", "negative", "neutral"]
#     df = df[cols]

#     return df




## Use these two functions - they're better and give more data than the above. 
# Sentiment calculation based on compound score
def get_sentiment(score):
    """
    Calculates the sentiment based on the compound score.
    """
    result = 0  # Neutral by default
    if score >= 0.05:  # Positive
        result = 1
    elif score <= -0.05:  # Negative
        result = -1

    return result

def return_sentiment_df_2(news_en_df):
    # Sentiment scores dictionaries
    title_sent = {
        "title_compound": [],
        "title_pos": [],
        "title_neu": [],
        "title_neg": [],
        "title_sent": [],
    }
    text_sent = {
        "text_compound": [],
        "text_pos": [],
        "text_neu": [],
        "text_neg": [],
        "text_sent": [],
    }

    # Get sentiment for the text and the title
    for index, row in news_en_df.iterrows():
        try:
            # Sentiment scoring with VADER
            title_sentiment = analyzer.polarity_scores(row["title"])
            title_sent["title_compound"].append(title_sentiment["compound"])
            title_sent["title_pos"].append(title_sentiment["pos"])
            title_sent["title_neu"].append(title_sentiment["neu"])
            title_sent["title_neg"].append(title_sentiment["neg"])
            title_sent["title_sent"].append(get_sentiment(title_sentiment["compound"]))

            text_sentiment = analyzer.polarity_scores(row["text"])
            text_sent["text_compound"].append(text_sentiment["compound"])
            text_sent["text_pos"].append(text_sentiment["pos"])
            text_sent["text_neu"].append(text_sentiment["neu"])
            text_sent["text_neg"].append(text_sentiment["neg"])
            text_sent["text_sent"].append(get_sentiment(text_sentiment["compound"]))
        except AttributeError:
            pass

        # Attaching sentiment columns to the News DataFrame
    title_sentiment_df = pd.DataFrame(title_sent)
    text_sentiment_df = pd.DataFrame(text_sent)
    news_en_df = news_en_df.join(title_sentiment_df).join(text_sentiment_df)

    return news_en_df


In [11]:
# Create the Bitcoin sentiment scores DataFrame
# YOUR CODE HERE!
#btc_sentiments = []
# btc_news_en

btc_sentiments = return_sentiment_df_2(btc_news_df)
btc_sentiments.head()

,title,description,text,date,language,title_compound,title_pos,title_neu,title_neg,title_sent,text_compound,text_pos,text_neu,text_neg,text_sent
0,"Who Bought $1.6B in Bitcoin Wednesday, and Why?",last week the cryptocurrency market persistent...,"Specifically, why did someone make a massive p...",2021-10-10,en,0.0000,0.000,1.000,0.000,0,0.5461,0.121,0.879,0.000,1
1,The Currency Of The Future: A Bitcoin (BTC) Boom,Bitcoin reaches a new all-time high as crypto-...,Following SEC Chair Gary Gensler's landmark ap...,2021-10-20,en,0.0000,0.000,1.000,0.000,0,0.5994,0.198,0.745,0.057,1
2,Crypto miners are hoarding bitcoin amid recent...,Long-term bitcoin holders just keep holding as...,Bitcoin. Sean Gladwell\r\nSean Gladwell\r\nCry...,2021-10-28,en,0.6124,0.314,0.579,0.107,1,0.4391,0.157,0.769,0.074,1
3,SEC Finally Greenlights Bitcoin Futures ETF Wi...,"Friday, October 16 saw the approval of a bitco...","Friday, October 16 saw the approval of a bitco...",2021-10-16,en,0.0000,0.000,1.000,0.000,0,0.4767,0.091,0.909,0.000,1
4,Bitcoin inflows hit record high so far in 2021...,Inflows into bitcoin products and funds have h...,"NEW YORK, Nov 8 (Reuters) - Inflows into bitco...",2021-11-08,en,0.0000,0.000,1.000,0.000,0,0.3612,0.072,0.928,0.000,1


In [12]:
# Create the Ethereum sentiment scores DataFrame
# YOUR CODE HERE!
eth_sentiments = return_sentiment_df_2(eth_news_df)
eth_sentiments.head()

,title,description,text,date,language,title_compound,title_pos,title_neu,title_neg,title_sent,text_compound,text_pos,text_neu,text_neg,text_sent
0,Nervos launches cross-chain bridge to connect ...,A new cross-chain bridge is currently connecte...,A new cross-chain bridge is currently connecte...,2021-10-16,en,0.0,0.0,1.0,0.0,0,0.0000,0.000,1.000,0.0,0
1,Cryptocurrency ether hits all time high of $44...,Ether the world's second largest cryptocurren...,"The exchange rates and logos of Bitcoin (BTH),...",2021-10-29,en,0.0,0.0,1.0,0.0,0,0.0000,0.000,1.000,0.0,0
2,Ethereum’s upgrade is finally coming — will it...,The world’s second most valuable cryptocurrenc...,The worlds second most valuable cryptocurrency...,2021-10-25,en,0.0,0.0,1.0,0.0,0,0.6478,0.150,0.850,0.0,1
3,On the road of 100 Ethereum Icon Renderings — ...,On the road of 100 Ethereum Icon Renderings — ...,Taoliang Wang is a visual designer based in Ch...,2021-10-12,en,0.0,0.0,1.0,0.0,0,0.2960,0.061,0.939,0.0,1
4,Wyoming Blockchain City Planned by Crypto Inve...,Blockchain entrepreneurs are now poised to bui...,Blockchain entrepreneurs are now poised to bui...,2021-11-10,en,0.0,0.0,1.0,0.0,0,0.2500,0.062,0.938,0.0,1


In [13]:
# Describe the Bitcoin Sentiment
# YOUR CODE HERE!
btc_sentiments.describe()

,title_compound,title_pos,title_neu,title_neg,title_sent,text_compound,text_pos,text_neu,text_neg,text_sent
count,20.000000,20.000000,20.00000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,0.088470,0.059650,0.91650,0.023850,0.150000,0.279835,0.085950,0.889150,0.024900,0.550000
std,0.241203,0.111906,0.13583,0.073385,0.587143,0.324864,0.064229,0.068692,0.032629,0.759155
min,-0.318200,0.000000,0.57900,0.000000,-1.000000,-0.440400,0.000000,0.745000,0.000000,-1.000000
25%,0.000000,0.000000,0.84850,0.000000,0.000000,0.000000,0.046500,0.844000,0.000000,0.000000
50%,0.000000,0.000000,1.00000,0.000000,0.000000,0.361200,0.077500,0.894000,0.000000,1.000000
75%,0.050575,0.032500,1.00000,0.000000,0.250000,0.543250,0.128000,0.936000,0.053250,1.000000
max,0.718400,0.316000,1.00000,0.315000,1.000000,0.726900,0.198000,1.000000,0.086000,1.000000


In [14]:
# Describe the Ethereum Sentiment
# YOUR CODE HERE!
eth_sentiments.describe()

,title_compound,title_pos,title_neu,title_neg,title_sent,text_compound,text_pos,text_neu,text_neg,text_sent
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.00000,20.00000,20.000000
mean,0.035105,0.024050,0.966950,0.009000,0.050000,0.184475,0.066100,0.91215,0.02180,0.400000
std,0.177568,0.076183,0.083475,0.040249,0.394034,0.332355,0.057487,0.06916,0.04145,0.820783
min,-0.296000,0.000000,0.704000,0.000000,-1.000000,-0.383900,0.000000,0.74500,0.00000,-1.000000
25%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.030000,0.86200,0.00000,0.000000
50%,0.000000,0.000000,1.000000,0.000000,0.000000,0.176350,0.057500,0.92900,0.00000,1.000000
75%,0.000000,0.000000,1.000000,0.000000,0.000000,0.473275,0.091000,0.95325,0.01325,1.000000
max,0.636900,0.296000,1.000000,0.180000,1.000000,0.647800,0.198000,1.00000,0.12500,1.000000


### Questions:

Q: Which coin had the highest mean positive score?

A: Bitcoin had the highest mean postive score. 

Q: Which coin had the highest compound score?

A: Bitcoin also had the higest compound score. 

Q. Which coin had the highest positive score?

A: Bitcoin has the higest postive score. 

---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [16]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [ ]:
# Instantiate the lemmatizer
# YOUR CODE HERE!
lemmatizer = WordNetLemmatizer()
# Create a list of stopwords
# YOUR CODE HERE!
sw = set(stopwords.words('english'))
# Expand the default stopwords list if necessary
# YOUR CODE HERE!

In [ ]:
# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
    
    # Remove the punctuation from text
    
   
    # Create a tokenized list of the words
    
    
    # Lemmatize words into root words

   
    # Convert the words to lowercase
    
    
    # Remove the stop words
    
    
    return tokens

In [ ]:
# Create a new tokens column for Bitcoin
# YOUR CODE HERE!

In [ ]:
# Create a new tokens column for Ethereum
# YOUR CODE HERE!

---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [ ]:
from collections import Counter
from nltk import ngrams

In [ ]:
# Generate the Bitcoin N-grams where N=2
# YOUR CODE HERE!

In [ ]:
# Generate the Ethereum N-grams where N=2
# YOUR CODE HERE!

In [ ]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=3):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [ ]:
# Use token_count to get the top 10 words for Bitcoin
# YOUR CODE HERE!

In [ ]:
# Use token_count to get the top 10 words for Ethereum
# YOUR CODE HERE!

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [ ]:
# Generate the Bitcoin word cloud
# YOUR CODE HERE!

In [ ]:
# Generate the Ethereum word cloud
# YOUR CODE HERE!

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [ ]:
import spacy
from spacy import displacy

In [ ]:
# Download the language model for SpaCy
# !python -m spacy download en_core_web_sm

In [ ]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [ ]:
# Concatenate all of the Bitcoin text together
# YOUR CODE HERE!

In [ ]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [ ]:
# Render the visualization
# YOUR CODE HERE!

In [ ]:
# List all Entities
# YOUR CODE HERE!

---

### Ethereum NER

In [ ]:
# Concatenate all of the Ethereum text together
# YOUR CODE HERE!

In [ ]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [ ]:
# Render the visualization
# YOUR CODE HERE!

In [ ]:
# List all Entities
# YOUR CODE HERE!

---